In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from glob import glob
import seaborn as sns
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import xarray as xr
import rasterio as rio
import rioxarray
import math
from sklearn.preprocessing import RobustScaler
from sklearn.decomposition import PCA
import pandas as pd
import os

import deep_snow.models
from deep_snow.dataset import norm_dict
from deep_snow.utils import calc_norm, undo_norm, calc_dowy
from tqdm import tqdm
import shutil

# 1. Dataset Preparation and Exploration (10%)

#### AI-Ready Data Utilization (4%): Demonstrates the use of the previously prepared AI-ready dataset effectively, ensuring consistency in preprocessing across models.

In [2]:
# we see the directory below contains our ai-ready data
files = glob("/mnt/c/Users/Jacke/uw/courses/aut24/ml_geo/jack_subsets/ncs/*.nc")
len(files)

3540

In [3]:
# now, let's open one of these files and understand the variables, shapes, and types of our data
ds = xr.open_dataset(files[0])
ds

<xarray.Dataset> Size: 1MB
Dimensions:    (x: 128, y: 128, samples: 16384)
Dimensions without coordinates: x, y, samples
Data variables: (12/14)
    aso_sd     (x, y) float32 66kB ...
    fcf        (x, y) float64 131kB ...
    elevation  (x, y) float32 66kB ...
    slope      (x, y) float64 131kB ...
    tri        (x, y) float64 131kB ...
    tpi        (x, y) float64 131kB ...
    ...         ...
    s1_pc1     (samples) float32 66kB ...
    s1_pc2     (samples) float32 66kB ...
    s2_pc1     (samples) float32 66kB ...
    s2_pc2     (samples) float32 66kB ...
    s2_pc3     (samples) float32 66kB ...
    dowy       (x, y) int64 131kB ...

In [4]:
# we see above that all of our files are 128x128 rasters (50m spatial resolution) where each cell has 14 variables
# note that the data here (besides the PCAs) are unscaled, the scaling happens in the data pipeline into the CNN 
# architectures as seen in section 3
# the data is below:

1. Target
    - aso_sd (snow depth measured from aerial LiDAR)
2. Spatial Attributes
    - latitude (lat of respective 50x50m cell)
    - longitude (lon of respective 50x50m cell)
3. Temporal Attribute
    - dowy (day of water year when LiDAR was collected)
3. Topography Data
    - elevation (m above egm2008 geoid)
    - slope (degrees)
    - tpi (topographic position index)
    - tri (topographic roughness index)    
4. Radar-derived Data
    - s1_pc1 (Principal component 1 of PCA of Sentinel-1 backscatter)
    - s1_pc2 (Principal component 2 of PCA of Sentinel-1 backscatter)
5. Optical-imagery-derived data
    - s2_pc1 (Principal component 1 of PCA of Sentinel-2 bands)
    - s2_pc2 (Principal component 2 of PCA of Sentinel-2 bands)
    - s2_pc3 (Principal component 3 of PCA of Sentinel-2 bands)

#### Exploratory Data Analysis (EDA) (3%): Includes visualizations and summaries to understand data distribution, temporal/spatial features, or domain-specific nuances.

In [ ]:
# given the nature of remote sensing data and pipelines to CNNs it's harder to visualized our AI-data this way
# so we'll just plot some stats from our raw data

#### Problem Setup (3%): Clearly defines the problem (e.g., regression/classification) and aligns the data with deep learning requirements (e.g., reshaping for CNNs, sequence creation for RNNs).

# 2. Model Benchmarking Against CML (10%)

#### Baseline Models (5%): Reports results from previous classical machine learning benchmarks (e.g., random forests, SVMs, or gradient boosting) with minimal additional work.

#### Performance Comparison (5%): Provides a high-level comparison of CML methods to deep learning models using relevant metrics (e.g., accuracy, RMSE, F1-score).

# 3. Model Architecture Exploration (35%)

#### Implementation and Justification (8%): Implements at least three deep learning architectures (e.g., FCN, CNN, RNN, U-Net). Justifies architecture choice based on dataset and problem type.

In [5]:
# first, let's just create a basic data pipeline to feed into a standard CNN
from torch.utils.data import DataLoader, TensorDataset, random_split
from deep_snow.dataset import norm_dict
from deep_snow.utils import calc_norm, undo_norm

In [10]:
norm_dict = {'aso_sd':[0, 25],
             'vv':[-59, 30],
             'vh':[-65, 17],
             'cr':[-43, 16],
             'delta_cr':[-33, 27],
             'AOT':[0, 572],
             'coastal':[0, 24304],
             'blue':[0, 23371],
             'green':[0, 26440],
             'red':[0, 21576],
             'red_edge1':[0, 20796],
             'red_edge2':[0, 20432],
             'red_edge3':[0, 20149],
             'nir':[0, 21217],
             'water_vapor':[0, 18199],
             'swir1':[0, 17669],
             'swir2':[0, 17936],
             'scene_class_map':[0, 15],
             'water_vapor_product':[0, 6518],
             'elevation':[-100, 9000],
             'aspect':[0, 360],
             'slope':[0, 90],
             'curvature':[-22, 22],
             'tpi':[-164, 167],
             'tri':[0, 913],
             'latitude':[-90, 90],
             'longitude':[-180, 180],
             'dowy': [0, 365]}

In [11]:
# Process each file and normalize the relevant features
def process_file(file_path):
    ds = xr.open_dataset(file_path)
    # Normalize features using the norm_dict
    data_dict = {}
    data_dict['latitude'] = calc_norm(torch.Tensor(ds['latitude'].values), norm_dict['latitude'])
    data_dict['longitude'] = calc_norm(torch.Tensor(ds['longitude'].values), norm_dict['longitude'])
    data_dict['elevation'] = calc_norm(torch.Tensor(ds['elevation'].values), norm_dict['elevation'])
    data_dict['slope'] = calc_norm(torch.Tensor(ds['slope'].values), norm_dict['slope'])
    data_dict['tri'] = calc_norm(torch.Tensor(ds['tri'].values), norm_dict['tri'])
    data_dict['tpi'] = calc_norm(torch.Tensor(ds['tpi'].values), norm_dict['tpi'])
    data_dict['dowy'] = calc_norm(torch.Tensor(ds['dowy'].values), norm_dict['dowy'])
    # Reshape PC components to 2D
    s1_pc1_2d = ds['s1_pc1'].values.reshape(128, 128)
    s1_pc2_2d = ds['s1_pc2'].values.reshape(128, 128)
    s2_pc1_2d = ds['s2_pc1'].values.reshape(128, 128)
    s2_pc2_2d = ds['s2_pc2'].values.reshape(128, 128)
    s2_pc3_2d = ds['s2_pc3'].values.reshape(128, 128)
    # Stack normalized features
    features = np.stack([
        data_dict['elevation'].numpy(),
        data_dict['slope'].numpy(),
        data_dict['tri'].numpy(),
        data_dict['tpi'].numpy(),
        data_dict['latitude'].numpy(),
        data_dict['longitude'].numpy(),
        s1_pc1_2d,
        s1_pc2_2d,
        s2_pc1_2d,
        s2_pc2_2d,
        s2_pc3_2d,
        data_dict['dowy'].numpy()
    ], axis=0)
    target = ds['aso_sd'].values
    return features, target

# Create datasets and data loaders
def create_dataset(file_list):
    features_list = []
    targets_list = []
    for file in file_list:
        features, target = process_file(file)
        features_list.append(torch.FloatTensor(features))
        targets_list.append(torch.FloatTensor(target))
    features_tensor = torch.stack(features_list)
    targets_tensor = torch.stack(targets_list)
    return TensorDataset(features_tensor, targets_tensor)

In [42]:
import random
# Split files into train, test, and validation sets
files_subset = files[:1280]
random.shuffle(files_subset)
# Calculate the sizes for each split
train_size = int(0.7 * len(files_subset))  # 70% for training
test_size = int(0.2 * len(files_subset))   # 20% for testing
val_size = len(files_subset) - train_size - test_size  # Remaining 10% for validation
# Create the splits
train_files = files_subset[:train_size]
test_files = files_subset[train_size:train_size + test_size]
val_files = files_subset[train_size + test_size:]
# Create datasets
train_dataset = create_dataset(train_files)
test_dataset = create_dataset(test_files)
val_dataset = create_dataset(val_files)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [43]:
# Print shapes for training loader
for features, targets in train_loader:
    print("Training batch shapes:")
    print(f"Features: {features.shape}")
    print(f"Targets: {targets.shape}")
    break
# Print shapes for test loader
for features, targets in test_loader:
    print("\nTest batch shapes:")
    print(f"Features: {features.shape}")
    print(f"Targets: {targets.shape}")
    break
# Print shapes for validation loader
for features, targets in val_loader:
    print("\nValidation batch shapes:")
    print(f"Features: {features.shape}")
    print(f"Targets: {targets.shape}")
    break

Training batch shapes:
Features: torch.Size([32, 12, 128, 128])
Targets: torch.Size([32, 128, 128])

Test batch shapes:
Features: torch.Size([32, 12, 128, 128])
Targets: torch.Size([32, 128, 128])

Validation batch shapes:
Features: torch.Size([32, 12, 128, 128])
Targets: torch.Size([32, 128, 128])


In [44]:
class SnowDepthCNN(nn.Module):
    def __init__(self):
        super(SnowDepthCNN, self).__init__()
        # First conv layer: (12, 128, 128) -> (32, 128, 128)
        self.conv1 = nn.Conv2d(in_channels=12, out_channels=32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        # Second conv layer: (32, 128, 128) -> (16, 128, 128)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=16, kernel_size=3, padding=1)
        # Final conv layer: (16, 128, 128) -> (1, 128, 128)
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=1, kernel_size=1)
    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.conv3(x)
        return x.squeeze(1)  # Remove channel dimension to match target shape

In [45]:
def train_and_evaluate_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs, device):
    """
    Train, validate, and test the model.

    Args:
        model (nn.Module): The PyTorch model to train and evaluate.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        test_loader (DataLoader): DataLoader for the test dataset.
        criterion (Loss): The loss function.
        optimizer (Optimizer): The optimizer for model training.
        num_epochs (int): Number of training epochs.
        device (torch.device): Device to use for computation.

    Returns:
        tuple: A tuple containing training losses, validation losses, and final test loss.
    """
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for features, targets in tqdm(train_loader, desc=f'Epoch {epoch+1}'):
            features, targets = features.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, targets in val_loader:
                features, targets = features.to(device), targets.to(device)
                outputs = model(features)
                val_loss += criterion(outputs, targets).item()
        
        # Compute average losses
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        print(f'Epoch [{epoch+1}/{num_epochs}]')
        print(f'Training Loss: {avg_train_loss:.4f}')
        print(f'Validation Loss: {avg_val_loss:.4f}')
    
    # Testing
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for features, targets in test_loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features)
            test_loss += criterion(outputs, targets).item()

    final_test_loss = test_loss / len(test_loader)
    print(f'\nFinal Test Loss: {final_test_loss:.4f}')

    return train_losses, val_losses, final_test_loss

In [46]:
# Initialize model
model = SnowDepthCNN()
# Loss function: MAE is appropriate for continuous regression problems like snow depth
criterion = nn.L1Loss()
# Optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
# Call the function
train_losses, val_losses, test_loss = train_and_evaluate_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

Epoch 1: 100%|██████████| 28/28 [00:05<00:00,  4.87it/s]


Epoch [1/10]
Training Loss: 0.7114
Validation Loss: 0.8744


Epoch 2: 100%|██████████| 28/28 [00:02<00:00,  9.91it/s]


Epoch [2/10]
Training Loss: 0.5730
Validation Loss: 0.4092


Epoch 3: 100%|██████████| 28/28 [00:05<00:00,  5.12it/s]


...

Epoch [8/10]
Training Loss: 0.3326
Validation Loss: 0.3159


Epoch 9: 100%|██████████| 28/28 [00:05<00:00,  4.83it/s]


Epoch [9/10]
Training Loss: 0.3308
Validation Loss: 0.3131


Epoch 10: 100%|██████████| 28/28 [00:06<00:00,  4.32it/s]


Epoch [10/10]
Training Loss: 0.3288
Validation Loss: 0.3118

Final Test Loss: 0.3423


In [ ]:
# great, we have a basic CNN implemented
# now let's explore other architectures for a preliminary search for the best architecture for our problem

In [30]:
class UNet(nn.Module):
    def __init__(self, in_channels=12, out_channels=1):
        super(UNet, self).__init__()
        # Encoding path
        self.enc1 = self.contract_block(in_channels, 64, 3)
        self.enc2 = self.contract_block(64, 128, 3)
        # Bottleneck
        self.bottleneck = self.contract_block(128, 256, 3)
        # Decoding path
        self.dec2 = self.expand_block(256, 128, 3)
        self.dec1 = self.expand_block(128, 64, 3)
        # Final output layer
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)
    def contract_block(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )

    def expand_block(self, in_channels, out_channels, kernel_size):
        return nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size, padding=1),
            nn.ReLU(),
            nn.Conv2d(out_channels, out_channels, kernel_size, padding=1),
            nn.ReLU(),
            nn.ConvTranspose2d(out_channels, out_channels, kernel_size=2, stride=2)
        )
    def forward(self, x):
        # Encoding
        enc1 = self.enc1(x)
        enc2 = self.enc2(enc1)
        bottleneck = self.bottleneck(enc2)
        dec2 = self.dec2(bottleneck)
        dec1 = self.dec1(dec2)
        output = self.final(dec1)
        # Resize the output to match the input size
        return F.interpolate(output, size=(x.size(2), x.size(3)), mode='bilinear', align_corners=False)

In [31]:
model = UNet().to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10
train_losses, val_losses, test_loss = train_and_evaluate_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

Epoch 1:   0%|          | 0/175 [00:00<?, ?it/s]

/home/jehayes/mambaforge/envs/deep-snow/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([4, 128, 128])) that is different to the input size (torch.Size([4, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)
Epoch 1: 100%|██████████| 175/175 [01:00<00:00,  2.89it/s]


Epoch [1/10]
Training Loss: 0.3626
Validation Loss: 0.3293


Epoch 2: 100%|██████████| 175/175 [00:59<00:00,  2.95it/s]


Epoch [2/10]
Training Loss: 0.3560
Validation Loss: 0.3282


...

Epoch [8/10]
Training Loss: 0.3561
Validation Loss: 0.3304


Epoch 9: 100%|██████████| 175/175 [01:04<00:00,  2.71it/s]


Epoch [9/10]
Training Loss: 0.3559
Validation Loss: 0.3284


Epoch 10: 100%|██████████| 175/175 [00:59<00:00,  2.92it/s]


Epoch [10/10]
Training Loss: 0.3556
Validation Loss: 0.3281

Final Test Loss: 0.3663


In [ ]:
# hmm doesn't seem to be doing particularly well
# granted this is a prelim test and batch sizes, model hyprerparameters, etc are all arbitrary
# and we're also using less than 1/3 of our total data

In [34]:
# our third architecture to explore will be a resnet
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size, padding=1)
        self.relu = nn.ReLU()
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size, padding=1)
        self.shortcut = (
            nn.Conv2d(in_channels, out_channels, kernel_size=1)
            if in_channels != out_channels
            else nn.Identity()
        )
    def forward(self, x):
        shortcut = self.shortcut(x)
        x = self.relu(self.conv1(x))
        x = self.conv2(x)
        return self.relu(x + shortcut)
class ResNetSnowDepth(nn.Module):
    def __init__(self):
        super(ResNetSnowDepth, self).__init__()
        # Initial convolution: (12, 128, 128) -> (32, 128, 128)
        self.initial_conv = nn.Conv2d(12, 32, kernel_size=3, padding=1)
        self.relu = nn.ReLU()
        # Residual blocks
        self.res1 = ResidualBlock(32, 64)  # (32, 128, 128) -> (64, 128, 128)
        self.res2 = ResidualBlock(64, 128)  # (64, 128, 128) -> (128, 128, 128)
        self.res3 = ResidualBlock(128, 64)  # (128, 128, 128) -> (64, 128, 128)
        # Output layer: (64, 128, 128) -> (1, 128, 128)
        self.output_conv = nn.Conv2d(64, 1, kernel_size=1)
    def forward(self, x):
        x = self.relu(self.initial_conv(x))
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.output_conv(x)
        return x.squeeze(1)  # Remove channel dimension to match target shape

In [35]:
model = ResNetSnowDepth()
model = model.to(device)
criterion = nn.L1Loss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
train_losses, val_losses, test_loss = train_and_evaluate_model(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=num_epochs,
    device=device
)

Epoch 1: 100%|██████████| 175/175 [02:12<00:00,  1.32it/s]


Epoch [1/10]
Training Loss: 0.4977
Validation Loss: 0.3482


Epoch 2: 100%|██████████| 175/175 [02:10<00:00,  1.34it/s]


Epoch [2/10]
Training Loss: 0.3936
Validation Loss: 1.0543


Epoch 3: 100%|██████████| 175/175 [02:13<00:00,  1.31it/s]


...

Epoch [8/10]
Training Loss: 0.3618
Validation Loss: 1.0612


Epoch 9: 100%|██████████| 175/175 [02:13<00:00,  1.31it/s]


Epoch [9/10]
Training Loss: 0.3908
Validation Loss: 0.3164


Epoch 10: 100%|██████████| 175/175 [02:13<00:00,  1.31it/s]


Epoch [10/10]
Training Loss: 0.3425
Validation Loss: 0.3267

Final Test Loss: 0.3646


In [ ]:
# above we see that all models perform similarly, but the simplest CNN ran in 53 seconds while the Unet ran in 11 minutes
# and the resnet ran in 23, so we'll proceed with a simple CNN
# this prelim analysis is not robust and limited by computational resources, but for the sake of the project we'll proceed

#### Parameter Tuning (8%): Explores hyperparameters (e.g., learning rate, number of layers, filter sizes) and documents experiments systematically.

In [57]:
# let's change our train and eval model function to get rid of print statements
def train_and_evaluate_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs, device, verbose=True):
    """
    Train, validate, and test the model.

    Args:
        model (nn.Module): The PyTorch model to train and evaluate.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        test_loader (DataLoader): DataLoader for the test dataset.
        criterion (Loss): The loss function.
        optimizer (Optimizer): The optimizer for model training.
        num_epochs (int): Number of training epochs.
        device (torch.device): Device to use for computation.
        verbose (bool): Whether to print detailed progress for each epoch.

    Returns:
        tuple: A tuple containing training losses, validation losses, and final test loss.
    """
    train_losses = []
    val_losses = []

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0
        for features, targets in train_loader:
            features, targets = features.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, targets in val_loader:
                features, targets = features.to(device), targets.to(device)
                outputs = model(features)
                val_loss += criterion(outputs, targets).item()
        
        # Compute average losses
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        if verbose:
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Training Loss: {avg_train_loss:.4f}')
            print(f'Validation Loss: {avg_val_loss:.4f}')
    
    # Testing
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for features, targets in test_loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features)
            test_loss += criterion(outputs, targets).item()

    final_test_loss = test_loss / len(test_loader)

    if verbose:
        print(f'\nFinal Test Loss: {final_test_loss:.4f}')

    return train_losses, val_losses, final_test_loss


In [38]:
import itertools
# Define the hyperparameter grid
learning_rates = [0.001, 0.0005, 0.0001]
num_layers_options = [3, 4, 5]
filter_sizes_options = [16, 32, 64]
# Results storage
tuning_results = []
# Define a function to create the model with variable layers and filter sizes
def create_model(num_layers, initial_filter_size):
    layers = []
    in_channels = 12
    for i in range(num_layers):
        out_channels = initial_filter_size if i == 0 else in_channels // 2
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    # Add the final layer
    layers.append(nn.Conv2d(in_channels, 1, kernel_size=1))
    return nn.Sequential(*layers)

# Train and evaluate models with different hyperparameters
for lr, num_layers, filter_size in itertools.product(learning_rates, num_layers_options, filter_sizes_options):
    print(f"\nTraining with learning rate={lr}, num_layers={num_layers}, filter_size={filter_size}")
    # Create the model
    model = create_model(num_layers, filter_size)
    model = model.to(device)
    # Define loss function and optimizer
    criterion = nn.L1Loss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    # Train the model
    train_losses, val_losses, test_loss = train_and_evaluate_model(
        model=model,
        train_loader=train_loader,
        val_loader=val_loader,
        test_loader=test_loader,
        criterion=criterion,
        optimizer=optimizer,
        num_epochs=num_epochs,
        device=device,
        verbose=False,  # Suppress progress for each epoch
    )
    # Record the results
    tuning_results.append({
        "learning_rate": lr,
        "num_layers": num_layers,
        "filter_size": filter_size,
        "train_loss": train_losses[-1],
        "val_loss": val_losses[-1],
        "test_loss": test_loss,
    })
# Display the tuning results
print("\nHyperparameter Tuning Results:")
for result in tuning_results:
    print(result)



Training with learning rate=0.001, num_layers=3, filter_size=16


/home/jehayes/mambaforge/envs/deep-snow/lib/python3.12/site-packages/torch/nn/modules/loss.py:128: UserWarning: Using a target size (torch.Size([4, 128, 128])) that is different to the input size (torch.Size([4, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.l1_loss(input, target, reduction=self.reduction)



Training with learning rate=0.001, num_layers=3, filter_size=32

Training with learning rate=0.001, num_layers=3, filter_size=64

Training with learning rate=0.001, num_layers=4, filter_size=16

Training with learning rate=0.001, num_layers=4, filter_size=32

Training with learning rate=0.001, num_layers=4, filter_size=64

Training with learning rate=0.001, num_layers=5, filter_size=16

Training with learning rate=0.001, num_layers=5, filter_size=32

Training with learning rate=0.001, num_layers=5, filter_size=64

Training with learning rate=0.0005, num_layers=3, filter_size=16

Training with learning rate=0.0005, num_layers=3, filter_size=32

Training with learning rate=0.0005, num_layers=3, filter_size=64

Training with learning rate=0.0005, num_layers=4, filter_size=16

Training with learning rate=0.0005, num_layers=4, filter_size=32

Training with learning rate=0.0005, num_layers=4, filter_size=64

Training with learning rate=0.0005, num_layers=5, filter_size=16

Training with lea

In [39]:
tuning_results_df = pd.DataFrame(tuning_results)
tuning_results_df.sort_values(by='test_loss', ascending=True).reset_index(drop=True)

,learning_rate,num_layers,filter_size,train_loss,val_loss,test_loss
0,0.0005,4,16,0.355223,0.328018,0.366206
1,0.0001,5,32,0.355208,0.328031,0.366213
2,0.0005,5,32,0.355295,0.328033,0.366221
3,0.0010,5,16,0.355255,0.328052,0.366241
4,0.0005,3,16,0.355256,0.328095,0.366284
5,0.0010,3,64,0.355376,0.328100,0.366294
6,0.0010,3,32,0.355271,0.328108,0.366298
7,0.0010,4,16,0.355412,0.328252,0.366434
8,0.0005,3,64,0.355377,0.328287,0.366465
9,0.0010,5,64,0.355474,0.328278,0.366469


In [40]:
# we see above that we'll proceed with a learning rate of 0.0005, 4 layers, and a filter size of 16

#### Incorporation of Physics-Informed Loss (4%): Implements physics-informed loss where appropriate, with a clear explanation of its relevance to the geoscientific problem.

In [54]:
class PhysicsInformedLoss(torch.nn.Module):
    def __init__(self, lambda_gradient=0.1):
        super(PhysicsInformedLoss, self).__init__()
        self.lambda_gradient = lambda_gradient

    def forward(self, predicted, target, features):
        """
        Compute the physics-informed loss.

        Args:
            predicted (Tensor): Predicted snow depth of shape (batch_size, 1, H, W)
            target (Tensor): Actual snow depth of shape (batch_size, H, W)
            features (Tensor): Features tensor of shape (batch_size, 12, H, W) including TPI

        Returns:
            Tensor: The computed loss value.
        """
        # Traditional MAE loss (mean absolute error)
        mae_loss = F.l1_loss(predicted, target)
        # Extract TPI from the features tensor
        tpi = features[:, 3, :, :]  # TPI is the 4th channel as seen from our data preprocessing
        # Calculate gradients of predicted snow depth and target snow depth
        grad_predicted = torch.abs(predicted[:, :, 1:, :] - predicted[:, :, :-1, :])  # Gradient in the x-direction
        # If target is 3D (batch_size, height, width), add a channel dimension
        target = target.unsqueeze(1)  # Adds a channel dimension (batch_size, 1, height, width)
        # Then the gradient calculation can proceed with the added channel dimension:
        grad_target = torch.abs(target[:, :, 1:, :] - target[:, :, :-1, :])  # Gradient in the x-direction
        # Compute terrain gradient loss considering TPI and elevation
        grad_tpi = torch.abs(tpi[:, 1:, :] - tpi[:, :-1, :])  # Gradient of TPI in x-direction
        terrain_gradient_loss = torch.mean(torch.abs(grad_predicted - grad_target) * grad_tpi)  # Weighted by TPI gradient
        # Total loss: MAE loss + weighted terrain gradient loss
        total_loss = mae_loss + self.lambda_gradient * terrain_gradient_loss
        return total_loss

Physics-informed loss explanation (ChatGPT's, I need to research this on my own and understand it better and what lambda to use):

Explanation of Physics-Informed Loss:


Traditional MAE Loss:

This term captures the basic prediction accuracy, comparing the predicted snow depth with the true snow depth at each pixel.


Terrain Gradient Loss:

We compute the gradient of both the predicted and actual snow depth with respect to neighboring pixels (in both the x and y directions).
The terrain gradient loss ensures that the model doesn’t produce unrealistic sharp transitions in snow depth where there should be smooth changes, especially considering the relationship between snow depth and elevation. This helps to model the spatial continuity of snow depth, which should naturally change with elevation and other terrain features.


Weighting Factor 𝜆

The weighting factor λ controls the importance of the gradient loss relative to the traditional MAE loss. This allows you to balance between fitting the data and adhering to the physical constraints.


Relevance to the Geoscientific Problem:

Snow Accumulation and Melting: Snow depth predictions should adhere to physical laws such as the conservation of mass. Sharp, discontinuous changes in snow depth are not physically realistic, especially when they occur across similar elevations.


Terrain Influence: 
Elevation plays a key role in snow accumulation and melting. The higher the elevation, the greater the snow accumulation. The physics-informed loss ensures that the snow depth predictions are consistent with the terrain features, making the model more physically grounded.


Improved Generalization: 
By adding domain knowledge into the loss function, the model is guided to produce more physically realistic predictions. This can help prevent overfitting and lead to better generalization in real-world applications, where snow depth predictions are needed for tasks like avalanche forecasting, hydrological modeling, or climate studies.

In [55]:
class SnowDepthCNN(nn.Module):
    def __init__(self):
        super(SnowDepthCNN, self).__init__()
        # Define the 4 layers with filter size 16
        self.conv1 = nn.Conv2d(in_channels=12, out_channels=16, kernel_size=3, padding=1)  # 12 input channels (from features)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)  # 16 output channels
        self.conv3 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)  # 16 output channels
        self.conv4 = nn.Conv2d(in_channels=16, out_channels=16, kernel_size=3, padding=1)  # 16 output channels
        # Fully connected layer (input size should be 16 * 8 * 8 = 1024)
        self.fc1 = nn.Linear(16 * 8 * 8, 512)  # Flattened output from conv layers: 16*8*8 = 1024
        self.fc2 = nn.Linear(512, 128 * 128)  # Output size to match the target (128x128)
    def forward(self, x):
        # Pass input through convolutional layers with ReLU activations and max pooling
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)  # Max pooling
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)  # Max pooling
        x = F.relu(self.conv3(x))
        x = F.max_pool2d(x, 2)  # Max pooling
        x = F.relu(self.conv4(x))
        x = F.max_pool2d(x, 2)  # Max pooling
        # Flatten the output for the fully connected layer
        x = x.view(x.size(0), -1)  # Flatten the tensor to (batch_size, 16 * 8 * 8 = 1024)
        # Pass through fully connected layers
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        # Reshape back to (batch_size, 1, 128, 128) for snow depth prediction
        x = x.view(-1, 1, 128, 128)
        return x

In [63]:
import torch
import json

def train_and_evaluate_model(model, train_loader, val_loader, test_loader, criterion, optimizer, num_epochs, device, verbose=True, save_dir='weights'):
    """
    Train, validate, and test the model. Save model weights, logs, and metrics to JSON.

    Args:
        model (nn.Module): The PyTorch model to train and evaluate.
        train_loader (DataLoader): DataLoader for the training dataset.
        val_loader (DataLoader): DataLoader for the validation dataset.
        test_loader (DataLoader): DataLoader for the test dataset.
        criterion (Loss): The loss function.
        optimizer (Optimizer): The optimizer for model training.
        num_epochs (int): Number of training epochs.
        device (torch.device): Device to use for computation.
        verbose (bool): Whether to print detailed progress for each epoch.
        save_dir (str): Directory to save model weights, logs, and metrics.

    Returns:
        tuple: A tuple containing training losses, validation losses, and final test loss.
    """
    train_losses = []
    val_losses = []
    performance_metrics = {}

    # Make sure the save directory exists
    os.makedirs(save_dir, exist_ok=True)

    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0
        for features, targets in train_loader:
            features, targets = features.to(device), targets.to(device)
            optimizer.zero_grad()
            outputs = model(features)
            loss = criterion(outputs, targets, features)  # Pass features as well
            loss.backward()
            optimizer.step()
            train_loss += loss.item()
        
        # Validation phase
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for features, targets in val_loader:
                features, targets = features.to(device), targets.to(device)
                outputs = model(features)
                val_loss += criterion(outputs, targets, features).item()  # Pass features as well
        
        # Compute average losses
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)

        if verbose:
            print(f'Epoch [{epoch+1}/{num_epochs}]')
            print(f'Training Loss: {avg_train_loss:.4f}')
            print(f'Validation Loss: {avg_val_loss:.4f}')
        
        # Save logs after each epoch (optional)
        performance_metrics[f"epoch_{epoch+1}"] = {
            "train_loss": avg_train_loss,
            "val_loss": avg_val_loss
        }

    # Testing phase
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for features, targets in test_loader:
            features, targets = features.to(device), targets.to(device)
            outputs = model(features)
            test_loss += criterion(outputs, targets, features).item()  # Pass features as well

    final_test_loss = test_loss / len(test_loader)
    performance_metrics["final_test_loss"] = final_test_loss

    if verbose:
        print(f'\nFinal Test Loss: {final_test_loss:.4f}')
    
    # Save model weights to file
    model_weight_path = os.path.join(save_dir, 'jackk_pinn1.pth')
    torch.save(model.state_dict(), model_weight_path)

    # Save training logs and performance metrics to JSON file
    performance_metrics_path = os.path.join(save_dir, 'performance_metrics.json')
    with open(performance_metrics_path, 'w') as json_file:
        json.dump(performance_metrics, json_file, indent=4)

    return train_losses, val_losses, final_test_loss

In [61]:
model = SnowDepthCNN()  # Your SnowDepthCNN model
loss_fn = PhysicsInformedLoss(lambda_gradient=0.1)  # Physics-Informed Loss function
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Adam optimizer
# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Train and evaluate the model using your custom function
train_losses, val_losses, test_loss = train_and_evaluate_model(
    model, train_loader, val_loader, test_loader, 
    criterion=loss_fn, optimizer=optimizer, 
    num_epochs=10, device=device, verbose=True
)

/tmp/ipykernel_5485/118312693.py:19: UserWarning: Using a target size (torch.Size([32, 128, 128])) that is different to the input size (torch.Size([32, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mae_loss = F.l1_loss(predicted, target)


Epoch [1/10]
Training Loss: 0.3760
Validation Loss: 0.3191
Epoch [2/10]
Training Loss: 0.3338
Validation Loss: 0.3138
Epoch [3/10]
Training Loss: 0.3308
Validation Loss: 0.3129
Epoch [4/10]
Training Loss: 0.3303
Validation Loss: 0.3126
Epoch [5/10]
Training Loss: 0.3305
Validation Loss: 0.3125
Epoch [6/10]
Training Loss: 0.3301
Validation Loss: 0.3124
Epoch [7/10]
Training Loss: 0.3298
Validation Loss: 0.3123
Epoch [8/10]
Training Loss: 0.3298
Validation Loss: 0.3123
Epoch [9/10]
Training Loss: 0.3298
Validation Loss: 0.3123
Epoch [10/10]
Training Loss: 0.3298
Validation Loss: 0.3123

Final Test Loss: 0.3441


In [62]:
files_subset = files[:]
random.shuffle(files_subset)
train_size = int(0.7 * len(files_subset))  # 70% for training
test_size = int(0.2 * len(files_subset))   # 20% for testing
val_size = len(files_subset) - train_size - test_size  # Remaining 10% for validation
train_files = files_subset[:train_size]
test_files = files_subset[train_size:train_size + test_size]
val_files = files_subset[train_size + test_size:]
train_dataset = create_dataset(train_files)
test_dataset = create_dataset(test_files)
val_dataset = create_dataset(val_files)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [64]:
model = SnowDepthCNN()  # Your SnowDepthCNN model
loss_fn = PhysicsInformedLoss(lambda_gradient=0.1)  # Physics-Informed Loss function
optimizer = optim.Adam(model.parameters(), lr=0.0005)  # Adam optimizer
# Move the model to the appropriate device (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
# Train and evaluate the model using your custom function
train_losses, val_losses, test_loss = train_and_evaluate_model(
    model, train_loader, val_loader, test_loader, 
    criterion=loss_fn, optimizer=optimizer, 
    num_epochs=10, device=device, verbose=True
)

/tmp/ipykernel_5485/118312693.py:19: UserWarning: Using a target size (torch.Size([32, 128, 128])) that is different to the input size (torch.Size([32, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mae_loss = F.l1_loss(predicted, target)
/tmp/ipykernel_5485/118312693.py:19: UserWarning: Using a target size (torch.Size([14, 128, 128])) that is different to the input size (torch.Size([14, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mae_loss = F.l1_loss(predicted, target)
/tmp/ipykernel_5485/118312693.py:19: UserWarning: Using a target size (torch.Size([2, 128, 128])) that is different to the input size (torch.Size([2, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mae_loss = F.l1_loss(predicted, target)


Epoch [1/10]
Training Loss: 0.6477
Validation Loss: 0.7031
Epoch [2/10]
Training Loss: 0.6397
Validation Loss: 0.7024
Epoch [3/10]
Training Loss: 0.6364
Validation Loss: 0.7021
Epoch [4/10]
Training Loss: 0.6397
Validation Loss: 0.7020
Epoch [5/10]
Training Loss: 0.6434
Validation Loss: 0.7020
Epoch [6/10]
Training Loss: 0.6375
Validation Loss: 0.7020
Epoch [7/10]
Training Loss: 0.6404
Validation Loss: 0.7020
Epoch [8/10]
Training Loss: 0.6352
Validation Loss: 0.7021
Epoch [9/10]
Training Loss: 0.6404
Validation Loss: 0.7020
Epoch [10/10]
Training Loss: 0.6398
Validation Loss: 0.7021

Final Test Loss: 0.5963


/tmp/ipykernel_5485/118312693.py:19: UserWarning: Using a target size (torch.Size([4, 128, 128])) that is different to the input size (torch.Size([4, 1, 128, 128])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  mae_loss = F.l1_loss(predicted, target)


In [ ]:
# as expected, the model performse worse when trained on the full dataset
# this is expected due to the fact that the model was seeing very similar data in the first 1000 files
# when we expanded it to the entire dataset, the model got exposed to data in mountainous regions with
# different geological and physical properties, which it had not seen before
# so creating a blanket model for all of these regions is more difficult